In [1]:
import sys
sys.path.append("..")

In [2]:
from collections import defaultdict

from resources.utils import get_puzzle_input

### Part 1

The power level in a given fuel cell can be found through the following process:

```
Find the fuel cell's rack ID, which is its X coordinate plus 10.
Begin with a power level of the rack ID times the Y coordinate.
Increase the power level by the value of the grid serial number (your puzzle input).
Set the power level to itself multiplied by the rack ID.
Keep only the hundreds digit of the power level (so 12345 becomes 3; numbers with no hundreds digit become 0).
Subtract 5 from the power level.
For example, to find the power level of the fuel cell at 3,5 in a grid with serial number 8:

The rack ID is 3 + 10 = 13.
The power level starts at 13 * 5 = 65.
Adding the serial number produces 65 + 8 = 73.
Multiplying by the rack ID produces 73 * 13 = 949.
The hundreds digit of 949 is 9.
Subtracting 5 produces 9 - 5 = 4.
So, the power level of this fuel cell is 4.
```

Here are some more example power levels:

```
Fuel cell at  122,79, grid serial number 57: power level -5.
Fuel cell at 217,196, grid serial number 39: power level  0.
Fuel cell at 101,153, grid serial number 71: power level  4.
```

Your goal is to find the 3x3 square which has the largest total power. The square must be entirely within the 300x300 grid. Identify this square using the X,Y coordinate of its top-left fuel cell. For example:

For grid serial number 18, the largest total 3x3 square has a top-left corner of 33,45 (with a total power of 29); these fuel cells appear in the middle of this 5x5 region:

```
-2  -4   4   4   4
-4   4   4   4  -5
 4   3   3   4  -4
 1   1   2   4  -3
-1   0   2  -5  -2
```

For grid serial number 42, the largest 3x3 square's top-left is 21,61 (with a total power of 30); they are in the middle of this region:

```
-3   4   2   2   2
-4   4   3   3   4
-5   3   3   4  -4
 4   3   3   4  -3
 3   3   3  -5  -1
 ```
 
What is the X,Y coordinate of the top-left fuel cell of the 3x3 square with the largest total power?

Your puzzle input is 9435.

In [3]:
class FuelCell:
    def __init__(self, x, y, serial_number):
        self.x = x
        self.y = y
        self.serial_number = serial_number
        
    @property
    def rack_id(self):
        return self.x + 10

    @property
    def power_level(self):
        """For example, to find the power level of the fuel cell at 3,5 in a grid with serial number 8:

        The rack ID is 3 + 10 = 13.
        The power level starts at 13 * 5 = 65.
        Adding the serial number produces 65 + 8 = 73.
        Multiplying by the rack ID produces 73 * 13 = 949.
        The hundreds digit of 949 is 9.
        Subtracting 5 produces 9 - 5 = 4.
        So, the power level of this fuel cell is 4.
        """
        level = self.rack_id * self.y
        level += self.serial_number
        level *= self.rack_id
        level = level // 100 % 10
        level -= 5
        return level

    def __key(self):
        return (self.x, self.y, self.serial_number)

    def __hash__(self):
        return hash(self.__key())

    def __eq__(self, other):
        return isinstance(self, type(other)) and self.__key() == other.__key()
    
    def __repr__(self):
        return "<{},{}> ({})".format(*self.__key)
    
    
        

In [4]:
assert FuelCell(3, 5, 8).power_level == 4

In [5]:
class PowerGrid:
    def __init__(self, width, serial_number):
        self.width = width
        self.serial_number = serial_number
        self._precomputed = dict()

        self._precompute_single_cells()
        
    def _precompute_single_cells(self):
        for x in range(1, self.width + 1):
            for y in range(1, self.width + 1):
                power = FuelCell(x, y, self.serial_number).power_level
                key = x, y, 1
                self._precomputed[key] = power

    def max_power(self, min_width=1, max_width=None):
        max_width = max_width or self.width

        max_power = None
        most_powerful = None
        
        for width in range(min_width, max_width + 1):
            print('Examing width: {}'.format(width))
            for x in range(1, self.width - width + 2):
                for y in range(1, self.width - width + 2):
                    key = x, y, width
                    power = self.power_of_square(*key)
                    if max_power is None or power > max_power:
                        max_power = power
                        most_powerful = key

        return most_powerful

    def power_of_square(self, top_left_x, top_left_y, width):
        key = top_left_x, top_left_y, width
        
        if key in self._precomputed:
            return self._precomputed[key]
        
        if (top_left_x, top_left_y, width - 1) in self._precomputed:
            power = self._power_from_precomputed(top_left_x, top_left_y, width)
        else:
            power = self._power_from_scratch(top_left_x, top_left_y, width)

        # Cache
        self._precomputed[key] = power

        return power
        
    def _power_from_precomputed(self, top_left_x, top_left_y, width):
        power = self._precomputed[top_left_x, top_left_y, width-1]

        # Add in the sides from  the previously computed grid
        for x in range(top_left_x, top_left_x + width):
            power += self._precomputed[x, top_left_y + width - 1, 1]
            
        for y in range(top_left_y, top_left_y + width - 1):
            power += self._precomputed[top_left_x + width - 1, y, 1]
            
        return power
    
    def _power_from_scratch(self, top_left_x, top_left_y, width):
        power = 0
        for x in range(top_left_x, top_left_x + width):
            for y in range(top_left_y, top_left_y + width):
                power += self._precomputed[x, y, 1]
        
        return power
    

In [6]:
grid = PowerGrid(300, 18)
assert grid.max_power(min_width=3, max_width=3) == (33, 45, 3)

Examing width: 3


In [7]:
grid = PowerGrid(300, 42)
assert grid.max_power(min_width=3, max_width=3) == (21, 61, 3)

Examing width: 3


In [8]:
# Puzzle solution
puzzle_grid = PowerGrid(300, 9435)
grid.max_power(min_width=3, max_width=3)

Examing width: 3


(21, 61, 3)

### Part 2

You discover a dial on the side of the device; it seems to let you select a square of any size, not just 3x3. Sizes from 1x1 to 300x300 are supported.

Realizing this, you now must find the square of any size with the largest total power. Identify this square by including its size as a third parameter after the top-left coordinate: a 9x9 square with a top-left corner of 3,5 is identified as 3,5,9.

For example:

```
For grid serial number 18, the largest total square (with a total power of 113) is 16x16 and has a top-left corner of 90,269, so its identifier is 90,269,16.
For grid serial number 42, the largest total square (with a total power of 119) is 12x12 and has a top-left corner of 232,251, so its identifier is 232,251,12.
```

What is the X,Y,size identifier of the square with the largest total power?

In [9]:
%%time

print(puzzle_grid.max_power())

Examing width: 1
Examing width: 2
Examing width: 3
Examing width: 4
Examing width: 5
Examing width: 6
Examing width: 7
Examing width: 8
Examing width: 9
Examing width: 10
Examing width: 11
Examing width: 12
Examing width: 13
Examing width: 14
Examing width: 15
Examing width: 16
Examing width: 17
Examing width: 18
Examing width: 19
Examing width: 20
Examing width: 21
Examing width: 22
Examing width: 23
Examing width: 24
Examing width: 25
Examing width: 26
Examing width: 27
Examing width: 28
Examing width: 29
Examing width: 30
Examing width: 31
Examing width: 32
Examing width: 33
Examing width: 34
Examing width: 35
Examing width: 36
Examing width: 37
Examing width: 38
Examing width: 39
Examing width: 40
Examing width: 41
Examing width: 42
Examing width: 43
Examing width: 44
Examing width: 45
Examing width: 46
Examing width: 47
Examing width: 48
Examing width: 49
Examing width: 50
Examing width: 51
Examing width: 52
Examing width: 53
Examing width: 54
Examing width: 55
Examing width: 56
E

### Improvements

I originally created the fuel cell object second guessing that part 2 would extend the power calculation to change over time. Given the calculation is straight forward it's likely that you can do a vast number of calculations for the price of a dictionary lookup and caching early stages (and single cell lookups) would be better achieved with a function call.